In [1]:

import sys
import os
from numba import jit
import numpy as np
import pandas as pd

file_path = "chr1_2.gt.tsv"
region_length = 50
hom_score = 0.025
het_score = -0.975



In [2]:
if file_path.endswith('.csv') or file_path.endswith('.csv.gz'):
    data = pd.read_csv(file_path, header = 0)

elif file_path.endswith('.tsv') or file_path.endswith('.tsv.gz'):
    data = pd.read_csv(file_path, sep = "\t", header = 0)

data.columns = ["chr", "locus", "allele", "seq_depth"]

data = data.iloc[:10_000]

In [3]:

def get_chromosome_list(data):
    #get all chromosomes present
    chromosomes = data['chr'].unique()
    chr_list = chromosomes.tolist()
    
    # convert to seperated list for iteration later
    return chr_list

chr_list = get_chromosome_list(data)
#print(chr_list)


In [4]:
def get_regions(chr_list):
     #initiate list for results
    results = []
    #we can't have stretches spanning chromosomes, therefore we go through each chromosome separate
    #option would be to introduce a check if more than one chr is present, but don't think it makes a time difference?
    for chr in chr_list:
        chr_data = data.loc[data["chr"] == chr]
            
        end = chr_data.shape[0] #get nrows

        #set to first row for now
        count = 0
        start = 0

        
        for locus in range(0,end):

            if chr_data.loc[locus,"allele"] == "0|0" or chr_data.loc[locus,"allele"] == "1|1":
                count +=1
                
            else:
                if count != 0:
                    stretch = [chr, chr_data.iloc[start,1], chr_data.iloc[locus,1], count] 
                    count = 0
                    start = locus
                    results.append(stretch)
                else:
                    continue

    regions = pd.DataFrame(results, columns = ['chr', 'start', 'end', 'n_hom'])
    return regions

regions = get_regions(chr_list)

regions_filt = regions[regions["n_hom"] > region_length]

print(regions_filt)

     chr    start      end  n_hom
442    1  3849266  3874699     54
451    1  4065623  4092868     53
660    1  5234177  5246541     54


In [5]:

regions_filt = regions_filt.assign(start_score=regions_filt["n_hom"] * hom_score)

print(regions_filt)

     chr    start      end  n_hom  start_score
442    1  3849266  3874699     54        1.350
451    1  4065623  4092868     53        1.325
660    1  5234177  5246541     54        1.350


In [6]:
def get_score(data):
    data = data.assign(id = range(1, len(data) + 1))
    data['allele_score'] = data['allele'].apply(lambda x: hom_score if x in ["1|1","0|0"] else (het_score if x in ["1|0","0|1"] else x))
    data['allele_score'] = pd.to_numeric(data['allele_score'], errors='coerce')
    return data

data = get_score(data)
print(data)

      chr    locus allele  seq_depth     id  allele_score
0       1    55326    1|1      19099      1         0.025
1       1    55545    1|1      17058      2         0.025
2       1    61987    1|0      20876      3        -0.975
3       1    61989    1|0      20815      4        -0.975
4       1    69511    1|0     368387      5        -0.975
...   ...      ...    ...        ...    ...           ...
9995    1  7555945    1|1      23119   9996         0.025
9996    1  7555953    1|1      22703   9997         0.025
9997    1  7556175    1|0      20453   9998        -0.975
9998    1  7556370    1|1      21110   9999         0.025
9999    1  7556533    1|0      23955  10000        -0.975

[10000 rows x 6 columns]


In [7]:
def extend_regions():
    for chr in chr_list:
        
        chr_data = data.loc[data["chr"] == chr]

        chr_regions = regions_filt.loc[regions_filt["chr"] == chr]

        for reg in range(chr_regions.shape[0]):

            score = chr_regions.iloc[reg].loc["start_score"]
            start = chr_regions.iloc[reg].loc["start"]
            end = chr_regions.iloc[reg].loc["end"]
            n_hom = chr_regions.iloc[reg].loc["n_hom"]

            row_nr_start = chr_data.loc[chr_data['locus'] == start,'id'].iat[0]
            row_nr_end = chr_data.loc[chr_data['locus'] == end, 'id'].iat[0]
                
            n_het = 0

            while score > 0:
                row_nr_start += -1
                row_nr_end += 1
                # start_score = chr_data.filter(pl.col("row_nr") == row_nr_start).select("locus_score")[0,0]
                # end_score = chr_data.filter(pl.col("row_nr") == row_nr_end).select("locus_score")[0,0]
                start_score = chr_data.loc[chr_data['id'] == row_nr_start, 'allele_score'].iat[0]
                end_score = chr_data.loc[chr_data['id'] == row_nr_end, 'allele_score'].iat[0]


                if start_score <0 :
                    n_het += 1

                elif start_score >0 :
                    n_hom += 1
                        
                if end_score  <0 :
                    n_het += 1

                elif end_score >0 :     
                    n_hom += 1

                score += start_score
                score += end_score

                if score <0:
                    if start_score <0 :
                        final_start = chr_data.loc[chr_data['id'] == row_nr_start +1, 'locus'].iat[0]
                    elif start_score >0 :
                        final_start = chr_data.loc[chr_data['id'] == row_nr_start, 'locus'].iat[0]

                    if end_score  <0 :
                        final_end = chr_data.loc[chr_data['id'] == row_nr_end -1, 'locus'].iat[0]
                    elif end_score >0 :
                        final_end = chr_data.loc[chr_data['id'] == row_nr_end, 'locus'].iat[0]

                    
                    stretch = [chr, final_start, final_end, n_hom, n_het] 
                    results_extended.append(stretch)

    regions_extended = np.DataFrame(results_extended, columns = ["chr", "start", "end", "n_hom", "n_het"])
    return regions_extended

results_extended = []
regions_extended = extend_regions()
print(regions_extended)

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [25]:

#for reg in range(chr_regions.shape[0]):
reg = 1

score = chr_regions.iloc[reg].loc["start_score"]
start = chr_regions.iloc[reg].loc["start"]
end = chr_regions.iloc[reg].loc["end"]
n_hom = chr_regions.iloc[reg].loc["n_hom"]

row_nr_start = chr_data.loc[chr_data['locus'] == start,'id'].iat[0]
row_nr_end = chr_data.loc[chr_data['locus'] == end, 'id'].iat[0]

print(score, row_nr_start)
n_het = 0


1.3250000000000002 4500


In [ ]:

    while score > 0:
        row_nr_start += -1
        row_nr_end += 1
        # start_score = chr_data.filter(pl.col("row_nr") == row_nr_start).select("locus_score")[0,0]
        # end_score = chr_data.filter(pl.col("row_nr") == row_nr_end).select("locus_score")[0,0]
        start_score = chr_data.loc[chr_data['id'] == row_nr_start, 'allele_score'].iat[0]
        end_score = chr_data.loc[chr_data['id'] == row_nr_end, 'allele_score'].iat[0]


        if start_score <0 :
            n_het += 1

        elif start_score >0 :
            n_hom += 1
                
        if end_score  <0 :
            n_het += 1

        elif end_score >0 :     
            n_hom += 1

        score += start_score
        score += end_score

        if score <0:
            if start_score <0 :
                final_start = chr_data.loc[chr_data['id'] == row_nr_start +1, 'locus'].iat[0]
            elif start_score >0 :
                final_start = chr_data.loc[chr_data['id'] == row_nr_start, 'locus'].iat[0]

            if end_score  <0 :
                final_end = chr_data.loc[chr_data['id'] == row_nr_end -1, 'locus'].iat[0]
            elif end_score >0 :
                final_end = chr_data.loc[chr_data['id'] == row_nr_end, 'locus'].iat[0]

            
            stretch = [chr, final_start, final_end, n_hom, n_het] 
            results_extended.append(stretch)

# regions_extended = np.DataFrame(results_extended, columns = ["chr", "start", "end", "n_hom", "n_het"])
# return regions_extended

results_extended = []
regions_extended = extend_regions()
print(regions_extended)

In [ ]:
  

regions_filt = regions_filt.with_columns([
    (np.col("n_hom") * hom_score).alias("start_score")])



results_extended = []
regions_extended = extend_regions()
           
regions_extended.write_csv("results_extended.csv")

